In [42]:
import tensorflow as tf
import pandas as pd
import numpy as np
train = pd.read_csv('trainnew.csv', sep=',')
valid = pd.read_csv('validnew.csv', sep=',')
train = train.sample(frac=1)
valid = valid.sample(frac=1)
print(list(train.head()))
len(list(train.head()))


['dist_walk_mid', 'dist_walk_low', 'dist_walk_high', 'no_ride', 'ride', 'hour6-12', 'hour0-6', 'hour12-18', 'hour18-24', 'MIRAMAR', 'ERANGEL', 'party_size_1', 'party_size_4', 'party_size_2', 'team_placement', 'M416', 'SCAR-L', 'hour', 'scaled_dmg_per', 'scaled_player_assists', 'scaled_player_dbno', 'scaled_player_dist_walk', 'scaled_player_dmg', 'scaled_player_kills', 'rank_categorized']


25

In [43]:
def parse_labels_and_features(dataset):
  labels = dataset.iloc[:,24]
  features = dataset.iloc[:,[0,1,3,5,6,7,9,15,16,18,19,20,21,22,23]] # Features
  return labels, features
training_targets, training_examples = parse_labels_and_features(train)
validation_targets, validation_examples = parse_labels_and_features(valid)


In [44]:
training_targets=training_targets-1
validation_targets=validation_targets-1
num=training_targets.count()

In [45]:
t=training_targets
v=validation_targets
b=tf.one_hot(t,4)
c=tf.one_hot(v,4)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    t = sess.run(b)
    v = sess.run(c)
    print(t)
t=pd.DataFrame(t)
v=pd.DataFrame(v)

[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]


0          0
1          0
2          0
3          0
4          0
5          0
6          0
7          0
8          0
9          0
10         0
11         0
12         0
13         0
14         0
15         0
16         0
17         0
18         0
19         0
20         0
21         0
22         0
23         0
24         0
25         0
26         0
27         0
28         0
29         0
          ..
3646341    3
3646342    3
3646343    3
3646344    3
3646345    3
3646346    3
3646347    3
3646348    3
3646349    3
3646350    3
3646351    3
3646352    3
3646353    3
3646354    3
3646355    3
3646356    3
3646357    3
3646358    3
3646359    3
3646360    3
3646361    3
3646362    3
3646363    3
3646364    3
3646365    3
3646366    3
3646367    3
3646368    3
3646369    3
3646370    3
Name: rank_categorized, Length: 3646371, dtype: int64

In [47]:
# hyper-parameter
result=0
learning_rate = 0.0022
epochs = 5
batch_size = 10000
# placeholder
x = tf.placeholder(tf.float32, [None, 15])
# y=0-3
y = tf.placeholder(tf.float32, [None, 4])
# hidden layer => w, b
W1 = tf.Variable(tf.random_normal([15, 75], stddev=0.03), name='W1')
b1 = tf.Variable(tf.random_normal([75]), name='b1')
# output layer => w, b
W2 = tf.Variable(tf.random_normal([75, 4], stddev=0.03), name='W2')
b2 = tf.Variable(tf.random_normal([4]), name='b2')
# hidden layer
hidden_out = tf.add(tf.matmul(x, W1), b1)
hidden_out = tf.nn.leaky_relu(hidden_out)

# output
y_ = tf.nn.softmax(tf.add(tf.matmul(hidden_out, W2), b2))
y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)
cross_entropy = -tf.reduce_sum(y * tf.log(y_clipped))
# potimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy)
# init operator
init_op = tf.global_variables_initializer()

# accuracy
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# session
with tf.Session() as sess:
    # init
    sess.run(init_op)
    steps=int(num/batch_size)
    for i in range(steps):
        start = (i * batch_size) % num
        end = min(start+batch_size,num)
        sess.run([optimizer,cross_entropy],feed_dict={x:training_examples[start:end],y:t[start:end]})
        cur=sess.run(accuracy,feed_dict={x: validation_examples, y: v})
        result=max(result,cur)
        if i%50==0:
            print(cur)
            
print(result)
ret[learning_rate]=result
print(ret)
fin=max(fin,ret[max(ret,key=ret.get)])
fin

0.024348203
0.5478095
0.6573274
0.68290895
0.6836511
0.68459564
0.68527853
0.6853567
0.68636703
{0.01: 0.6910889, 0.012: 0.6913431, 0.014: 0.69134974, 0.016: 0.6915818, 0.018: 0.69176275, 0.02: 0.69167805, 0.022: 0.69265133, 0.024: 0.69169116, 0.026: 0.6907738, 0.028: 0.6920573, 0.03: 0.6887506, 0.033: 0.6920022, 0.036: 0.6915818, 0.005: 0.6925016, 0.006: 0.69243664, 0.004: 0.692486, 0.002: 0.690081, 0.0022: 0.68636703}


0.6930932

0.6921676
{0.01: 0.6910889, 0.012: 0.6913431, 0.014: 0.69134974, 0.016: 0.6915818, 0.018: 0.69176275, 0.02: 0.69167805, 0.022: 0.6921676}


0.6921676